In [1]:
import pandas as pd
import helpers
from importlib import reload
import random
import models
#import rocket
import numpy as np
import os
from datetime import datetime
from datetime import datetime, timezone

2024-06-12 20:29:40.545038: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 20:29:43.479079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Load pressure drop list

In [3]:
reload(helpers)

pressuredrops = pd.read_csv('allPdrops_ordered.csv').sort_values(by=' SOL ')

unique_sols = pressuredrops[' SOL '].unique()
missing_sols = []
for sol in unique_sols:
    file_path = f'seisdata_pq/sol_{sol}_seisdata.parquet'
    if not os.path.isfile(file_path):
        pressuredrops = pressuredrops[pressuredrops[' SOL '] != sol]
        missing_sols.append(sol)

pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '] = pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '].str.strip()
pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '] = pd.to_datetime(pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '], format='%Y-%jT%H:%M:%S.%fZ')

pressure_threshold = -0

filtered_drops = pressuredrops[pressuredrops['_DROP_ '] < pressure_threshold]
print(filtered_drops.describe())

            _DROP_        _LTST_           SOL   \
count  11560.000000  11560.000000  11560.000000   
mean      -0.611105     12.289125    460.845329   
min       -9.183000      8.001000     65.000000   
25%       -0.603000     10.890500    309.000000   
50%       -0.450000     12.193500    434.500000   
75%       -0.389000     13.601250    639.000000   
max       -0.345000     21.058000    861.000000   
std        0.511879      1.839304    191.826052   

            YYYY-MM-DDTHH:MM:SS.sss          RATIO  
count                          11560  11560.000000  
mean   2020-03-14 06:10:28.823963136      0.989166  
min       2019-02-01 11:13:59.915000      0.502000  
25%    2019-10-10 04:44:05.309999872      1.000000  
50%    2020-02-16 02:58:21.841500160      1.000000  
75%    2020-09-13 08:57:15.142499840      1.000000  
max       2021-04-29 12:06:01.663000      1.000000  
std                              NaN      0.044255  


### Feature List

In [3]:
feature_list = [
    #'Total',
    #'Mid frequency data',
    #'High frequency data'
    'Time of day',
    #'Time of year',
    #'Time since last pDrop',
    #'Drop width',
    #'Max power',
    #'Max/average',
    'Gradient'
]

### Extract dataset

In [ ]:
reload(helpers)
window_size = 35
features = 3
power_threshold = 0
samples_per_drop = 3
pos_sample_spacing = 5
sol_range = [14, 862]
test_sols = helpers.generate_test_sols(sol_range=sol_range, percentage=0.0)
all_df, pDrops_below_thresh, pressuredrops, pDrops_out_of_range, pos_thresh_dict = helpers.construct_pos_dataset(feature_list=feature_list, sol_range=sol_range, test_sols=test_sols, drop_list=filtered_drops, power_threshold=power_threshold, window_size = window_size, samples_per_drop = samples_per_drop, pos_sample_spacing= pos_sample_spacing, verbose=1, drop_magnitude=True)

### Concat features, create labels

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



rows_with_nan = all_df.isnull().any(axis=1)
indices_with_nan = all_df.index[rows_with_nan]
all_df.drop(indices_with_nan, inplace=True)
all_df.columns = all_df.columns.astype(str)
labels_df = all_df['Magnitude']
features_df = all_df.drop(columns=['Magnitude'])


assert len(features_df) == len(labels_df), f"Mismatch in length of features and labels dataframes, features length {len(features_df)}, labels length {len(labels_df)}"

feature_scaler = StandardScaler()
features_scaled = pd.DataFrame(feature_scaler.fit_transform(features_df), columns=features_df.columns, index=features_df.index)
if 'Time of day' in feature_list:
    mlst_series = features_scaled.index.to_series().apply(lambda x: helpers.datetime_to_mlst(x))
    features_scaled[['sin_time', 'cos_time']] = mlst_series.apply(helpers.cyclical_encode_time).apply(pd.Series)
if 'Time of year' in feature_list:
    features_scaled[['sin_martian_time_of_year', 'cos_martian_time_of_year']] = features_scaled.index.to_series().apply(helpers.cyclical_encode_martian_time_of_year).apply(pd.Series)

if 'Time since last pDrop' in feature_list:
    pressuredrops_time = pressuredrops[[' YYYY-MM-DDTHH:MM:SS.sss ']].sort_values(' YYYY-MM-DDTHH:MM:SS.sss ')
    features_scaled = features_scaled.sort_index()
    features_scaled = pd.merge_asof(features_scaled, pressuredrops_time, left_index=True, right_on=' YYYY-MM-DDTHH:MM:SS.sss ', direction='backward')
    features_scaled['time since last pDrop'] = features_scaled.index - features_scaled[' YYYY-MM-DDTHH:MM:SS.sss ']
    features_scaled['time since last pDrop'] = features_scaled['time since last pDrop'].dt.total_seconds().astype(int)
    features_scaled['time since last pDrop'] = features_scaled['time since last pDrop'].clip(upper=86400)
    features_scaled.drop(' YYYY-MM-DDTHH:MM:SS.sss ', axis=1, inplace=True)

X_train, X_eval, y_train, y_eval = train_test_split(features_scaled, labels_df, test_size=0.2, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_eval, y_eval, test_size=0.5, random_state=123)
X_train_final = pd.concat([X_train, X_val])
y_train_final = pd.concat([y_train, y_val])

train_data = [X_train, y_train, X_val, y_val]

### XGBoost

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
reload(models)

adjust_ratio = False

best_xgb_clf = models.train_xgboost_regressor(data=[df.copy() for df in train_data], bayes_search=True, verbose=1, ratio=0, adjust_ratio=adjust_ratio)
y_pred = best_xgb_clf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)
r2 = r2_score(y_test, y_pred)
print("R-squared (R²) Score:", r2)

best_xgb_clf.fit(X_train_final, y_train_final, eval_set=[(X_test, y_test)])


## Save Model and Scaler

In [7]:
import joblib
joblib.dump(best_xgb_clf, 'magnitude_clf.joblib')
joblib.dump(feature_scaler, 'magnitude_feature_scaler.joblib')

['magnitude_feature_scaler.joblib']

## Add Magnitudes to Classifier Predictions

In [ ]:
reload(helpers)
classifier = 'magnitude_clf.joblib'
feature_scaler = 'magnitude_feature_scaler.joblib'
directory = 'xgboost_feature_selection/xgb_0_1.148e-08_Time of day_Time of year'
feature_list = ['Time of Day', 'Gradient']
helpers.add_magnitude_predictions(filtered_drops, directory, classifier, 35, feature_scaler, feature_list)